In [1]:
import os


In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\LENOVO\\OneDrive\\Desktop\\Miraffra\\Internship-Mlops-Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    # alpha: float
    # l1_ratio: float
    target_column: str

In [5]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        params_filepath = PARAMS_FILE_PATH,
        config_filepath = CONFIG_FILE_PATH,
        
        schema_filepath = SCHEMA_FILE_PATH):

        self.params = read_yaml(params_filepath)
        self.config = read_yaml(config_filepath)
        
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        # params = self.params.LogisticRegression
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            # alpha = params.alpha,
            # l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )

        return model_trainer_config

In [7]:
import pandas as pd
import os
from mlProject import logger
# from sklearn.linear_model import ElasticNet
import joblib
from sklearn.linear_model import LogisticRegression
# from sklearn.naive_bayes import MultinomialNB



In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score



In [9]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE

from imblearn.under_sampling import RandomUnderSampler

from imblearn.over_sampling import ADASYN

from imblearn.under_sampling import NearMiss


from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier



from xgboost import XGBClassifier




class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_data = train_data.dropna(subset=[self.config.target_column])
        test_data = test_data.dropna(subset=[self.config.target_column])


        # Separate features and target
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        # Feature scaling
        # scaler = StandardScaler()
        scaler = MinMaxScaler()
        train_x_scaled = scaler.fit_transform(train_x)
        test_x_scaled = scaler.transform(test_x)

        # Handling class imbalance
        # smote = SMOTE(random_state=42)
        # train_x_balanced, train_y_balanced = smote.fit_resample(train_x_scaled, train_y)

        # rus = RandomUnderSampler(random_state=42)
        # train_x_balanced, train_y_balanced = rus.fit_resample(train_x_scaled, train_y)

        # adasyn = ADASYN(random_state=42)
        # train_x_balanced, train_y_balanced = adasyn.fit_resample(train_x_scaled, train_y)

        nearmiss = NearMiss()
        train_x_balanced, train_y_balanced = nearmiss.fit_resample(train_x_scaled, train_y)

        # Logistic Regression with class weights
        # lr = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)

        # Fit the model
        # lr.fit(train_x_balanced, train_y_balanced)

        # nb = MultinomialNB()
        # nb.fit(train_x_balanced, train_y_balanced)

        # Make predictions
        # predictions = lr.predict(test_x_scaled)

        # predictions = nb.predict(test_x_scaled)


        # xgb = XGBClassifier(
        #     n_estimators=100,
        #     scale_pos_weight=1,  # Set this to balance class weight
        #     random_state=42,
        #     eval_metric='logloss'
        # )
        # xgb.fit(train_x_balanced, train_y_balanced)

        # # Make predictions
        # predictions = xgb.predict(test_x_scaled)

        rf = RandomForestClassifier(n_estimators=100,class_weight='balanced', random_state=42)
        rf.fit(train_x_balanced, train_y_balanced)
        predictions = rf.predict(test_x_scaled)

        # gbc = GradientBoostingClassifier(n_estimators=100, random_state=42)
        # gbc.fit(train_x_balanced, train_y_balanced)
        # predictions = gbc.predict(test_x_scaled)

        # Calculate metrics
        accuracy = accuracy_score(test_y, predictions)
        precision = precision_score(test_y, predictions)
        recall = recall_score(test_y, predictions)
        f1 = f1_score(test_y, predictions)

        # Print metrics
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")

        # Save the model
        joblib.dump(rf, os.path.join(self.config.root_dir, self.config.model_name))

        # Logging
        logger.info("Model training completed and model saved.")
        # logger.info(f"Accuracy: {accuracy:.4f}")
        # logger.info(f"Precision: {precision:.4f}")
        logger.info(f"Recall: {recall:.4f}")
        logger.info(f"F1 Score: {f1:.4f}")

        print(f"Training and testing shapes: {train_x.shape}, {test_x.shape}")

        

        

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-07-29 12:10:09,110: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-29 12:10:09,113: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-29 12:10:09,116: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-29 12:10:09,116: INFO: common: created directory at: artifacts]
[2024-07-29 12:10:09,117: INFO: common: created directory at: artifacts/model_trainer]
Accuracy: 0.6041
Precision: 1.0000
Recall: 0.6041
F1 Score: 0.7532
[2024-07-29 12:10:16,969: INFO: 1763198257: Model training completed and model saved.]
[2024-07-29 12:10:16,969: INFO: 1763198257: Recall: 0.6041]
[2024-07-29 12:10:16,969: INFO: 1763198257: F1 Score: 0.7532]
Training and testing shapes: (3652, 5000), (245, 5000)
